In [23]:
#importing all the dependencies packages
import json
import csv
import nltk
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem     import PorterStemmer
from nltk.stem     import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import string
import re
import csv
import pandas as pd
import numpy as np
st = LancasterStemmer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))    
def extract_documents(sentence1,productName):
    #removing punctuations and stopwords to extract main relavant data
    document_list = []
    sentence1 = sentence1.lower() # lowercase text
    sentence1 = REPLACE_BY_SPACE_RE.sub(' ', sentence1) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    sentence1 = BAD_SYMBOLS_RE.sub('', sentence1) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    sentence1 = sentence1.replace('x', '')
    sentence1 = ' '.join(word for word in sentence1.split() if word not in STOPWORDS) # remove stopwords from text
    extracted_words_from_sentence = set()
    
    for key in end_json:
        for word in end_json[key]:
            pattern = r"\b" + re.escape(word) + r"\b"
            match = re.search(pattern, sentence1)
            if match:
                extracted_words_from_sentence.add(key)
#     print("words from sentence")
#     print(extracted_words_from_sentence)
    #comparisions based on products
    df = pd.read_excel (r'./test_sample_1.xlsx')
    data = df[['Keywords (m)','Product','Document Name','DOC ID']]
    data = data.sort_values(by=['Product'])
    data = data.dropna()
    data = data.replace(r'^\s*$', np.nan, regex=True)
    data['Product'] = data['Product'].apply(str.lower)
    data['extracted_words'] = str(extracted_words_from_sentence)
    data['Keywords (m)'] = data['Keywords (m)'].str.lower()
    data = data[data.Product.str.contains('none|n/a|no data|nan')==False]
    data = data[data.Product.str.contains(str(productName).lower())]
    data['new_keywords'] = ''
    data['score']=0.0
    #importing the keywords excel for comparision
    for sent in data['Keywords (m)']:
        t = re.sub(r'\s*,\s*',',',str(sent))
        data['Keywords (m)'] = data['Keywords (m)'].replace([sent],t)
    for idx1,sentence2 in data['Keywords (m)'].iteritems():
        string_words_new = ''
        extracted_words_from_key_words = set()
        if str(sentence2).lower() not in ('nan','n/a','none','no data'):
            for key in end_json:
                for word in end_json[key]:
                    new_sentence_split = np.array(sentence2.split(','))
                    if word in new_sentence_split:
                        extracted_words_from_key_words.add(key)
            for val in extracted_words_from_key_words:
                string_words_new = string_words_new+ ',' + val
            data.at[idx1, 'new_keywords'] = string_words_new
    for idx,sentence in data['new_keywords'].iteritems():
        if str(sentence).lower() not in ('nan','n/a','none','no data','','[none]'):
            array_toCompare = np.array(sentence.split(','))
            flt_sent2 = [w for w in array_toCompare if not w.lower() in STOPWORDS and w not in string.punctuation and w not in ('nan','n/a','none','no data','','[none]')]
            ac = [x.lower() for x in flt_sent2]
            common_elements = list(extracted_words_from_sentence.intersection(ac))
            union_elements = list(set(extracted_words_from_sentence).union(set(ac)))
            #Calculating the score
            score = len(common_elements)/len(union_elements)*100
            data.at[idx, 'score'] = score
    data.to_csv('pavan.csv')
    df_final = data.nlargest(3, 'score')
    result_final_output = df_final.to_json(orient="records")
    #result_final_json_output = json.loads(result_final_output)
    return result_final_output

In [10]:
output_sentence = extract_documents("Provider wants to know if novo needles contain nickel for patient that has nickel allergy Also, wants information on GLP-1 and gastroparesis in the setting of post gastric bypass in the setting of weight regain","General-Obesity")
print(output_sentence)

words from sentence
{'needles', 'gastric', 'post', 'information', 'regain', 'allergy', 'gastroparesis', 'bypass', 'weight', 'glp-1', 'gastric bypass'}
[]
[]


In [25]:
#to calculate accuracy of data
df_accuracy = pd.read_excel (r'./data_to_test.xlsx')
for idx_acc,sentence_acc in df_accuracy['Question'].iteritems() :
    df_accuracy.at[idx_acc, 'accuracy'] = 'not accurate'
    output_sentence = extract_documents(sentence_acc,df_accuracy.at[idx_acc,'Product'])
    df_accuracy.at[idx_acc, 'extracted_keywords_'+ str(count)] = dj['extracted_words']
    json_data = json.loads(output_sentence)
    count = 1
    for dj in json_data:
        if dj['DOC ID'] == df_accuracy.at[idx_acc,'DOC ID']:
            df_accuracy.at[idx_acc, 'accuracy'] = 'accurate with one of docID'
        df_accuracy.at[idx_acc, 'keywords_'+ str(count)] = dj['Keywords (m)']
        df_accuracy.at[idx_acc, 'Document_Name_'+ str(count)] = dj['Document Name']
        df_accuracy.at[idx_acc, 'DOC_ID_'+ str(count)] = dj['DOC ID']
        df_accuracy.at[idx_acc, 'new_keywords_'+ str(count)] = dj['new_keywords']
        df_accuracy.at[idx_acc, 'score_'+ str(count)] = dj['score']
        count+=1
df_accuracy.to_csv('accuracy.csv')

[{'Keywords (m)': 'overview,approval,summary', 'Product': 'wegovy', 'Document Name': 'Approval and Product Overview (SEOB 010) (v3.0)', 'DOC ID': 'SEOB 010', 'extracted_words': "{'dr', 'available', 'overview'}", 'new_keywords': ',summary,approval,overview', 'score': 20.0}, {'Keywords (m)': 'medicaid,formulary,summary,overview', 'Product': 'wegovy', 'Document Name': 'Wegovy Medicaid Summary (SEOB 400) (v2.0)', 'DOC ID': 'SEOB 400', 'extracted_words': "{'dr', 'available', 'overview'}", 'new_keywords': ',medicaid,summary,formulary,overview', 'score': 16.6666666667}, {'Keywords (m)': 'step,overview,slide,clinical,pi', 'Product': 'wegovy', 'Document Name': 'Clinical Overview of Wegovy (SEOB 601) (v1.0)', 'DOC ID': 'SEOB 601', 'extracted_words': "{'dr', 'available', 'overview'}", 'new_keywords': ',step,slide,clinical,pi,overview', 'score': 14.2857142857}]
[{'Keywords (m)': 'hypoglycemia,hypo,safety', 'Product': 'ozempic', 'Document Name': 'Hypoglycemia (SEM 120) (v9.0)', 'DOC ID': 'SEM 120',

[{'Keywords (m)': 'renal,kidney,outcomes,creatinine,uar,scr,glomerular,gfr', 'Product': 'gen glp-1, ozempic, rybelsus, victoza', 'Document Name': 'Renal Outcomes (GLP1 012) (v2.0)', 'DOC ID': 'GLP1 012', 'extracted_words': "{'level', 'overall', 'send', 'function', 'impact', 'creatinine', 'vs', 'hypoglycemia', 'kidney', 'information'}", 'new_keywords': ',creatinine,scr,outcomes,renal,gfr,kidney,uar,glomerular', 'score': 12.5}, {'Keywords (m)': 'renal,kidney,dialysis,esrd,gfr,egfr,creatinine,clearance,crcl,nephrotic,transplant', 'Product': 'ozempic', 'Document Name': 'Renal Impairment (SEM 125) (v13.0)', 'DOC ID': 'SEM 125', 'extracted_words': "{'level', 'overall', 'send', 'function', 'impact', 'creatinine', 'vs', 'hypoglycemia', 'kidney', 'information'}", 'new_keywords': ',creatinine,clearance,nephrotic,dialysis,renal,gfr,kidney,egfr,transplant,crcl,esrd', 'score': 10.5263157895}, {'Keywords (m)': 'hypoglycemia,hypo,safety', 'Product': 'ozempic', 'Document Name': 'Hypoglycemia (SEM 120)

[{'Keywords (m)': 'rationale,dosing,dose,qw,week', 'Product': 'wegovy', 'Document Name': 'Rationale for QW Dosing (SEOB 002) (v8.0)', 'DOC ID': 'SEOB 002', 'extracted_words': "{'move', 'week', 'send', 'mi', 'request'}", 'new_keywords': ',qw,rationale,week,dose', 'score': 12.5}, {'Keywords (m)': 'excursion', 'Product': 'all products, fiasp, gen glp-1, general, insulin portfolio, levemir, novolog, novolog mix 70/30, novolin 70/30, novolin n, novolin r, other, ozempic, saxenda, tresiba, victoza, wegovy, xultophy 100/3.6', 'Document Name': 'Cover to Excursion Data for Novo Nordisk Products (GEN 123) (v6.0)', 'DOC ID': 'GEN 123', 'extracted_words': "{'move', 'week', 'send', 'mi', 'request'}", 'new_keywords': ',excursion', 'score': 0.0}, {'Keywords (m)': 'bariatric surgery,anti-obesity medications,saxenda,wegovy', 'Product': 'gen obesity, saxenda, wegovy', 'Document Name': 'Bariatric Surgery (GOBS 001) (v1.0)', 'DOC ID': 'GOBS 001', 'extracted_words': "{'move', 'week', 'send', 'mi', 'request

[{'Keywords (m)': 'ozempic,semaglutide injection,drug,interaction', 'Product': 'ozempic', 'Document Name': 'Drug Interactions (SEM 158) (v10.0)', 'DOC ID': 'SEM 158', 'extracted_words': "{'study', 'ozempic', 'information'}", 'new_keywords': ',interaction,drug,semaglutide injection,ozempic', 'score': 16.6666666667}, {'Keywords (m)': 'ozempic,glucose,semaglutide,fasting,fpg,preprandial', 'Product': 'ozempic', 'Document Name': 'Fasting Plasma Glucose (FPG) (SEM 025) (v9.0)', 'DOC ID': 'SEM 025', 'extracted_words': "{'study', 'ozempic', 'information'}", 'new_keywords': ',fpg,ozempic,preprandial,glucose,fasting,semaglutide', 'score': 12.5}, {'Keywords (m)': 'ozempic,semaglutide injection,glycemic,control,0.25,initial,early,switching,initiation,hyperglycemia', 'Product': 'ozempic', 'Document Name': 'Initial Glycemic Control (SEM 021) (v7.0)', 'DOC ID': 'SEM 021', 'extracted_words': "{'study', 'ozempic', 'information'}", 'new_keywords': ',hyperglycemia,ozempic,semaglutide injection,glycemic,s

[{'Keywords (m)': 'renal,kidney,dialysis,esrd,gfr,egfr,creatinine,clearance,crcl,nephrotic,transplant', 'Product': 'ozempic', 'Document Name': 'Renal Impairment (SEM 125) (v13.0)', 'DOC ID': 'SEM 125', 'extracted_words': "{'2', 'type 2', 'ozempic', 'dialysis', 'egfr'}", 'new_keywords': ',creatinine,clearance,nephrotic,dialysis,renal,gfr,kidney,egfr,transplant,crcl,esrd', 'score': 14.2857142857}, {'Keywords (m)': 'ozempic,semaglutide injection,drug,interaction', 'Product': 'ozempic', 'Document Name': 'Drug Interactions (SEM 158) (v10.0)', 'DOC ID': 'SEM 158', 'extracted_words': "{'2', 'type 2', 'ozempic', 'dialysis', 'egfr'}", 'new_keywords': ',interaction,drug,semaglutide injection,ozempic', 'score': 12.5}, {'Keywords (m)': 'ozempic,glucose,semaglutide,fasting,fpg,preprandial', 'Product': 'ozempic', 'Document Name': 'Fasting Plasma Glucose (FPG) (SEM 025) (v9.0)', 'DOC ID': 'SEM 025', 'extracted_words': "{'2', 'type 2', 'ozempic', 'dialysis', 'egfr'}", 'new_keywords': ',fpg,ozempic,pre

...

[{'Keywords (m)': 'long-term,longest,duration,longer,long,step 5', 'Product': 'wegovy', 'Document Name': 'Long-Term Data (SEOB 088) (v3.0)', 'DOC ID': 'SEOB 088', 'extracted_words': "{'data', 'long', 'regain', 'longterm', 'dr', 'wegovy', 'therapy', 'long term', 'weight', 'longest', 'term'}", 'new_keywords': ',long,step 5,duration,longterm,long-term,longest', 'score': 21.4285714286}, {'Keywords (m)': 'step 5,long term,2 year,104 weeks,sustained', 'Product': 'wegovy', 'Document Name': 'Cover to STEP 5 (SEOB 105) (v2.0)', 'DOC ID': 'SEOB 105', 'extracted_words': "{'data', 'long', 'regain', 'longterm', 'dr', 'wegovy', 'therapy', 'long term', 'weight', 'longest', 'term'}", 'new_keywords': ',sustained,step 5,longterm,long term,2 year,104 weeks', 'score': 13.3333333333}, {'Keywords (m)': 'bariatric surgery,anti-obesity medications,saxenda,wegovy', 'Product': 'gen obesity, saxenda, wegovy', 'Document Name': 'Bariatric Surgery (GOBS 001) (v1.0)', 'DOC ID': 'GOBS 001', 'extracted_words': "{'data

[{'Keywords (m)': 'sustain,phase,trials,trial,clinical,overview', 'Product': 'ozempic', 'Document Name': 'SUSTAIN Overview (All Phase 3) (SEM 111) (v10.0)', 'DOC ID': 'SEM 111', 'extracted_words': "{'data', 'clinical', 'ozempic', 'trial', 'retinopathy', 'diabetic retinopathy', 'hcp', 'diabetic', 'trials', 'information'}", 'new_keywords': ',clinical,phase,trial,sustain,overview,trials', 'score': 23.0769230769}, {'Keywords (m)': 'retinopathy,eye,drc,diabetic retinopathy complications,diabetic retinopathy,blindness,blind,vitreous hemorrhage,retina,visual,microvascular', 'Product': 'ozempic', 'Document Name': 'Retinopathy (SEM 004) (v16.0)', 'DOC ID': 'SEM 004', 'extracted_words': "{'data', 'clinical', 'ozempic', 'trial', 'retinopathy', 'diabetic retinopathy', 'hcp', 'diabetic', 'trials', 'information'}", 'new_keywords': ',visual,microvascular,diabetic retinopathy,drc,blindness,blind,vitreous hemorrhage,retinopathy,eye,diabetic retinopathy complications,retina', 'score': 10.5263157895}, {'

[{'Keywords (m)': 'binge eating disorder,binge eating,bed,bulimia', 'Product': 'saxenda', 'Document Name': 'Binge Eating Disorder (SAX 093) (v7.0)', 'DOC ID': 'SAX 093', 'extracted_words': "{'3', 'treatment', 'obese', 'hcp', 'bed', 'disorder', 'information'}", 'new_keywords': ',bed,bulimia', 'score': 12.5}, {'Keywords (m)': 'summary,overview,information,monograph', 'Product': 'saxenda', 'Document Name': 'Product Overview (SAX 025) (v7.0)', 'DOC ID': 'SAX 025', 'extracted_words': "{'3', 'treatment', 'obese', 'hcp', 'bed', 'disorder', 'information'}", 'new_keywords': ',information,summary,overview,monograph', 'score': 10.0}, {'Keywords (m)': 'excursion', 'Product': 'all products, fiasp, gen glp-1, general, insulin portfolio, levemir, novolog, novolog mix 70/30, novolin 70/30, novolin n, novolin r, other, ozempic, saxenda, tresiba, victoza, wegovy, xultophy 100/3.6', 'Document Name': 'Cover to Excursion Data for Novo Nordisk Products (GEN 123) (v6.0)', 'DOC ID': 'GEN 123', 'extracted_word

[{'Keywords (m)': 'safety,rybelsus,ozempic,ae', 'Product': 'rybelsus', 'Document Name': 'Safety vs Ozempic (OSEM 143) (v5.0)', 'DOC ID': 'OSEM 143', 'extracted_words': "{'data', 'rybelsus', 'ozempic', 'use', 'semaglutide'}", 'new_keywords': ',ae,rybelsus,ozempic,safety', 'score': 28.5714285714}, {'Keywords (m)': 'cv,rybelsus,ozempic,semglutide,indication', 'Product': 'rybelsus', 'Document Name': 'Rybelsus vs Ozempic-CV Indication (OSEM 133) (v5.0)', 'DOC ID': 'OSEM 133', 'extracted_words': "{'data', 'rybelsus', 'ozempic', 'use', 'semaglutide'}", 'new_keywords': ',ozempic,rybelsus,semglutide,cardiovascular,indication', 'score': 25.0}, {'Keywords (m)': 'ozempic,comparison,semaglutide injection,semaglutide sq,semaglutide sc,semaglutide subcutaneous,semaglutide comparison', 'Product': 'rybelsus', 'Document Name': 'vs Ozempic (OSEM 040) (v16.0)', 'DOC ID': 'OSEM 040', 'extracted_words': "{'data', 'rybelsus', 'ozempic', 'use', 'semaglutide'}", 'new_keywords': ',semaglutide subcutaneous,ozemp

[{'Keywords (m)': 'cv,cardiovascular,mace,heart attack,mi,stroke,death,indication', 'Product': 'rybelsus', 'Document Name': 'CV Indication (OSEM 093) (v9.0)', 'DOC ID': 'OSEM 093', 'extracted_words': "{'reaction', 'send', 'mi', 'allergy'}", 'new_keywords': ',stroke,mace,mi,cardiovascular,indication,heart attack,death', 'score': 10.0}, {'Keywords (m)': 'allergic,allergy,hypersensitivity,sensitivity,anaphylaxis,anaphylactic,urticaria,rash,immunogenic,immunogenicity', 'Product': 'rybelsus', 'Document Name': 'Allergic Reactions (OSEM 044) (v8.0)', 'DOC ID': 'OSEM 044', 'extracted_words': "{'reaction', 'send', 'mi', 'allergy'}", 'new_keywords': ',immunogenic,hypersensitivity,anaphylaxis,anaphylactic,allergy,urticaria,sensitivity,rash', 'score': 9.0909090909}, {'Keywords (m)': 'niosh,hazardous', 'Product': 'gen glp-1, ozempic, rybelsus', 'Document Name': 'NIOSH Classification (Sema) (SEM 300) (v6.0)', 'DOC ID': 'SEM 300', 'extracted_words': "{'reaction', 'send', 'mi', 'allergy'}", 'new_keywo

In [93]:
df_tocheck = pd.read_csv(r'./accuracy.csv')
df_tocheck['accuracy'].value_counts()['accurate']
df_tocheck['accuracy'].count()
accuracy = df_tocheck['accuracy'].value_counts()['accurate']/df_tocheck['accuracy'].count()
print(accuracy)

0.3447228549734245


In [1]:
end_json = {
    "low": ["low","lowest","lower"],
    "low-dose": ["low-dose"],
    "0.25": ["0.25"],
    "initiation": ["initiation"],
    "starting dose": ["starting dose"],
    "cutoff": ["cutoff"],
    "2.8": ["2.8"],
    "sensitivity": ["sensitivity"],
    "specificity": ["specificity"],
    "3 mg": ["3 mg"],
    "start": ["start","starting","starter"],
    "usp": ["usp"],
    "797": ["797"],
    "sterile": ["sterile","sterility"],
    "stable": ["stable","stability"],
    "test": ["test"],
    "stress": ["stress"],
    "iso": ["iso"],
    "accurate": ["accurate","accuracy"],
    "volume": ["volume","volumes"],
    "international organization for standardization": [
        "international organization for standardization"
    ],
    "deliver": ["deliver","delivery"],
    "international": ["international"],
    "standardidization": ["standardidization"],
    "international organization": ["international organization"],
    "organization": ["organization"],
    "gi disease": ["gi disease"],
    "gi conditions": ["gi conditions"],
    "gerd": ["gerd"],
    "gastritis": ["gastritis"],
    "ulcerative colitis": ["ulcerative colitis"],
    "ibs": ["ibs","irritable bowel syndrome"],
    "crohn's disease": ["crohn's disease"],
    "acute": ["acute"],
    "chronic": ["chronic"],
    "gi": ["gi","gastrointestinal"],
    "colitis": ["colitis"],
    "crohn's": ["crohn's"],
    "adolescent": ["adolescent","adolescents"],
    "teens": ["teens"],
    "adverse event": ["adverse event","adverse events"],
    "reporting": ["reporting"],
    "fda": ["fda"],
    "adverse": ["adverse"],
    "events": ["events"],
    "safety": ["safe","safety"],
    "ae": ["ae"],
    "common": ["common"],
    "adverse reaction": ["adverse reaction"],
    "side effect": ["side effect"],
    "air": ["air"],
    "bubble": ["bubble","bubbles"],
    "foam": ["foam","foams"],
    "xray": ["xray","x-ray","x-rays","xrays"],
    "airport": ["airport"],
    "security": ["security"],
    "tsa": ["tsa","transportation security administration"],
    "transportation": ["transportation"],
    "radiation": ["radiation"],
    "albumin": ["albumin"],
    "hypoalbuminemia": ["hypoalbuminemia"],
    "protein": ["protein"],
    "alcohol": ["alcohol"],
    "alcohol use": ["alcohol use"],
    "alcohol dependency": ["alcohol dependency"],
    "consumption": ["consumption"],
    "pancreatitis": ["pancreatitis"],
    "amylase": ["amylase"],
    "lipase": ["lipase"],
    "allergy": ["allergic","allergies","allergy"],
    "amyloidosis": ["amyloidosis"],
    "hypersensitivity": ["hypersensitivity"],
    "angioedema": ["angioedema"],
    "anaphylactic": ["anaphylactic"],
    "anaphylaxis": ["anaphylaxis"],
    "reaction": ["reaction","reactions"],
    "skin": ["skin"],
    "iasp": ["iasp"],
    "nnpi": ["nnpi"],
    "unbranded biologic": ["unbranded biologic"],
    "urticaria": ["urticaria"],
    "rash": ["rash"],
    "immunogenic": ["immunogenic","immunogenicity"],
    "immune complex disease": ["immune complex disease"],
    "antibody": ["antibody","antibodies"],
    "injection": ["injection"],
    "contraindication": ["contraindication","contraindicated"],
    "niacin": ["niacin"],
    "alopecia": ["alopecia"],
    "hair": ["hair"],
    "hair loss": ["hair loss"],
    "alzheimer": ["alzheimer"],
    "dementia": ["dementia"],
    "cognition": ["cognition","cognitive"],
    "answer registry": ["answer registry"],
    "template": ["template"],
    "anti-liraglutide": ["anti-liraglutide","anti liraglutide"],
    "neutralizing": ["neutralizing"],
    "pharmacokinetics": ["pharmacokinetics","pk"],
    "pharmacology": ["pharmacology"],
    "absorption": ["absorption"],
    "distribution": ["distribution"],
    "metabolism": ["metabolism"],
    "excretion": ["excretion"],
    "elimination": ["elimination"],
    "half-life": ["half-life"],
    "steady": ["steady"],
    "step": ["step"],
    "phase": ["phase"],
    "trial": ["trial","trials"],
    "study": ["study","studies"],
    "program": ["program"],
    "approval": ["approval"],
    "available": ["available","availability"],
    "overview": ["overview"],
    "launch": ["launch"],
    "date": ["date"],
    "macrilen": ["macrilen"],
    "approval and product overview": ["approval and product overview"],
    "prescribing information": ["prescribing information"],
    "glp-1": ["glp-1","glp 1","glp1"],
    "summary": ["summary"],
    "bariatric surgery glp-1 ras victoza ozempic rybelsus": [
        "bariatric surgery glp-1 ras victoza ozempic rybelsus"
    ],
    "bariatric surgery": ["bariatric surgery"],
    "anti-obesity medications": ["anti-obesity medications"],
    "saxenda": ["saxenda"],
    "wegovy": ["wegovy"],
    "bariatric": ["bariatric"],
    "gastric bypass": ["gastric bypass"],
    "roux": ["roux"],
    "roux-en-y": ["roux-en-y"],
    "banding": ["banding"],
    "surgery": ["surgery"],
    "stapling": ["stapling","stapled"],
    "obesity-related surgery": ["obesity-related surgery","obesity related surgery"],
    "sleeve": ["sleeve"],
    "gastric": ["gastric"],
    "bypass": ["bypass"],
    "lap banding": ["lap banding"],
    "battery": ["battery"],
    "life": ["life"],
    "novopen echo": ["novopen echo"],
    "echo": ["echo"],
    "expire": ["expire"],
    "duration": ["duration"],
    "last": ["last"],
    "dead": ["dead"],
    "novopen": ["novopen"],
    "beta cell": ["beta cell","beta-cell"],
    "function": ["function"],
    "preservation": ["preservation"],
    "homa-b": ["homa-b","homab","homa b"],
    "c-peptide": ["c-peptide","c peptide"],
    "beta": ["beta"],
    "cell": ["cell"],
    "insulin": ["insulin"],
    "secretion": ["secretion"],
    "glucagon": ["glucagon"],
    "infusion": ["infusion"],
    "pancreas": ["pancreas"],
    "rate": ["rate"],
    "isr": ["isr"],
    "igtt": ["igtt"],
    "homa ir": ["homa ir","homa-ir","homair"],
    "homa": ["homa"],
    "proinsulin": ["proinsulin"],
    "apoptosis": ["apoptosis"],
    "apoptotic": ["apoptotic"],
    "bed": ["bed","binge eating disorder","binge eating"],
    "bulimia": ["bulimia"],
    "blood brain barrier": ["blood brain barrier","bbb","blood-brain-barrier","blood-brain barrier"],
    "bp": ["bp","blood pressure"],
    "pressure": ["pressure"],
    "systolic": ["systolic"],
    "sbp": ["sbp"],
    "diastolic": ["diastolic"],
    "dbp": ["dbp"],
    "hypertension": ["hypertension","hypertensive"],
    "pulse": ["pulse"],
    "hr": ["hr"],
    "blood": ["blood"],
    "heart": ["heart"],
    "cardiovascular": ["cardiovascular","cv"],
    "bpm": ["bpm"],
    "high blood pressure": ["high blood pressure","hbp"],
    "systolic blood pressure": ["systolic blood pressure","sbp"],
    "diastolic blood pressure": ["diastolic blood pressure","dbp"],
    "heart rate": ["heart rate","hr"],
    "body composition": ["body composition"],
    "obesity": ["obesity"],
    "type 2 diabetes": ["type 2 diabetes"],
    "fat tissue": ["fat tissue"],
    "lean tissue": ["lean tissue"],
    "visceral fat": ["visceral fat"],
    "subcutaneous fat": ["subcutaneous fat"],
    "fat": ["fat"],
    "lean": ["lean"],
    "visceral": ["visceral"],
    "dxa": ["dxa"],
    "dexa": ["dexa"],
    "body mass": ["body mass","body-mass"],
    "breast": ["breast"],
    "cancer": ["cancer"],
    "neoplasm": ["neoplasm","neoplasms"],
    "carcinoma": ["carcinoma","carcunimas"],
    "tumor": ["tumor","tumors"],
    "oncology": ["oncology"],
    "naive": ["naive"],
    "type 2": ["type 2","type-2","type2"],
    "amylin": ["amylin"],
    "carcinomia": ["carcinomia"],
    "carcinogen": ["carcinogen","carcinogenicity"],
    "tumors": ["tumors"],
    "carcinomas": ["carcinomas"],
    "arterial": ["arterial","artery","atrial"],
    "atrioventricular": ["atrioventricular"],
    "cardiac": ["cardiac","cardiology"],
    "coronary": ["coronary"],
    "cpb": ["cpb"],
    "implant": ["implant","implantation"],
    "lvad": ["lvad"],
    "septal": ["septal"],
    "valve": ["valve"],
    "ventricle": ["ventricle","ventricular"],
    "cabg": ["cabg"],
    "ecmo": ["ecmo"],
    "extracorporeal": ["extracorporeal"],
    "cardiopulmonary": ["cardiopulmonary"],
    "cardiothoracic": ["cardiothoracic"],
    "cvd":["cardiovascular disease","cvd"],
    "ascvd":["atherosclerotic cardiovascular disease","ascvd"],
    "mace":["major adverse cardiovascular events","mace"],
    "myocardial": ["myocardial"],
    "infarction": ["infarction","infarctions"],
    "mi": ["mi","myocardial infarction"],
    "heart attack": ["heart attack"],
    "stroke": ["stroke"],
    "ischemia": ["ischemia"],
    "ischemic": ["ischemic"],
    "cvot": ["cardiovascular outcome trial","cvot"],
    "outcome": ["outcome"],
    "select": ["select"],
    "sustain 6": ["sustain 6"],
    "heart failure": ["heart failure"],
    "hf": ["hf","hear failure"],
    "categorical": ["categorical"],
    "proportion": ["proportion"],
    "5%": ["5%"],
    "10%": ["10%"],
    "15%": ["15%"],
    "20%": ["20%"],
    "cgm": ["cgm"],
    "concomitant": ["concomitant"],
    "oad": ["obstructive airway disease","oad"],
    "antidiabetic": ["antidiabetic"],
    "diabetic": ["diabetic"],
    "lipid lowering": ["lipid lowering","lipid-lowering"],
    "antihypertensive": ["antihypertensive"],
    "medication": ["medication","medications"],
    "drug": ["drug"],
    "chrohn's disease": ["chrohn's disease"],
    "chrohn's": ["chrohn's"],
    "ulcer": ["ulcer"],
    "peptic": ["peptic"],
    "question": ["question"],
    "clarification": ["clarification"],
    "monograph": ["monograph"],
    "dossier": ["dossier"],
    "trials": ["trials"],
    "data": ["data"],
    "phase 3": ["phase 3","phase3","phase-3"],
    "clinical": ["clinical"],
    "slide": ["slide","slides"],
    "deck": ["deck"],
    "pi": ["pi"],
    "trial participation": ["trial participation"],
    "study participation": ["study participation"],
    "combination": ["combination","combinations","combine","combo","combining"],
    "two": ["two"],
    "duplicate": ["duplicate","duplicates"],
    "basal": ["basal"],
    "long-acting": ["long-acting","long acting"],
    "dpp4": ["dpp4","dpp-4i","dpp-4","dipeptidyl peptidase-4"],
    "sitagliptin": ["sitagliptin"],
    "dipeptidyl peptidase inhibitor": ["dipeptidyl peptidase inhibitor"],
    "gliptin": ["gliptin"],
    "dipeptydl peptidase": ["dipeptydl peptidase"],
    "januvia": ["januvia"],
    "saxagliptin": ["saxagliptin"],
    "onglyza": ["onglyza"],
    "linagliptin": ["linagliptin"],
    "tradjenta": ["tradjenta"],
    "fasting": ["fasting"],
    "linaclotide": ["linaclotide"],
    "bisphosphonate": ["bisphosphonate"],
    "fosamax": ["fosamax"],
    "linzess": ["linzess"],
    "synthroid": ["synthroid"],
    "levothyroxine": ["levothyroxine"],
    "feiba": ["feiba"],
    "apcc": ["apcc"],
    "sequential": ["sequential"],
    "hemlibra": ["hemlibra"],
    "emicizumab": ["emicizumab"],
    "roche": ["roche"],
    "fviii mimetic": ["fviii mimetic"],
    "concurrent": ["concurrent"],
    "scale insulin": ["scale insulin"],
    "glargine": ["glargine"],
    "degludec": ["degludec"],
    "tresiba": ["tresiba"],
    "nph": ["nph"],
    "lantus": ["lantus"],
    "aspart": ["aspart"],
    "mdi": ["mdi"],
    "transition-t2d": ["transition-t2d"],
    "detemir": ["detemir"],
    "glulisine": ["glulisine"],
    "lispro": ["lispro"],
    "levemir": ["levemir"],
    "basaglar": ["basaglar"],
    "toujeo": ["toujeo"],
    "novolog": ["novolog"],
    "humalog": ["humalog"],
    "apidra": ["apidra"],
    "fiasp": ["fiasp"],
    "niad": ["niad"],
    "oral": ["oral"],
    "oads": ["oads"],
    "metformin": ["metformin"],
    "sulfonylurea": ["sulfonylurea"],
    "dpp-4i": ["dpp-4i","dpp4i"],
    "tzd": ["tzd","tzds"],
    "thiazolidinedione": ["thiazolidinedione"],
    "contraception": ["contraception","contraceptive"],
    "birth control": ["birth control"],
    "discontinuation": ["discontinuation"],
    "reproduction": ["reproduction"],
    "reproduce": ["reproduce"],
    "pregnant": ["pregnant"],
    "pregnancy": ["pregnancy"],
    "antiobesity medications": ["antiobesity medications"],
    "phentermine": ["phentermine"],
    "qsymia": ["qsymia"],
    "liraglutide": ["liraglutide"],
    "weight loss": ["weight loss"],
    "aom": ["aom"],
    "contrave": ["contrave"],
    "bupropion": ["bupropion"],
    "naltrexone": ["naltrexone"],
    "topiramate": ["topiramate"],
    "orlistat": ["orlistat"],
    "xenical": ["xenical"],
    "anti-obesity": ["anti-obesity","anti obesity"],
    "alli": ["alli"],
    "ozempic": ["ozempic"],
    "victoza": ["victoza"],
    "trulicity": ["trulicity"],
    "bydureon": ["bydureon"],
    "byetta": ["byetta"],
    "adlyxin": ["adlyxin"],
    "dulaglutide": ["dulaglutide"],
    "exenatide": ["exenatide"],
    "lixisenatide": ["lixisenatide"],
    "bolus": ["bolus"],
    "prandial": ["prandial"],
    "mealtime": ["mealtime"],
    "short-acting": ["short-acting","short acting"],
    "rapid-acting": ["rapid-acting"],
    "sglt2": ["sglt2"],
    "jardiance": ["jardiance"],
    "empagliflozin": ["empagliflozin"],
    "sodium glucose co-transporter": ["sodium glucose co-transporter"],
    "7 mg": ["7 mg"],
    "14 mg": ["14 mg"],
    "compare": ["compare","compared","comparison"],
    "vs": ["vs"],
    "versus": ["versus"],
    "fix": ["fix"],
    "factor": ["factor"],
    "ix": ["ix"],
    "factor ix": ["factor ix"],
    "rfviii": ["rfviii"],
    "fviii": ["fviii"],
    "viii": ["viii"],
    "connected devices": ["connected devices"],
    "nni insulins": ["nni insulins"],
    "inpen": ["inpen"],
    "humapen luxura": ["humapen luxura"],
    "compound": ["compound"],
    "carnitine": ["carnitine"],
    "b12": ["b12"],
    "chf": ["chf"],
    "congestive": ["congestive"],
    "devote": ["devote"],
    "dialoq": ["dialoq"],
    "connected": ["connected"],
    "smart": ["smart"],
    "device": ["device"],
    "attachment": ["attachment"],
    "pen": ["pen"],
    "novopen 6": ["novopen 6"],
    "novopen echo plus": ["novopen echo plus"],
    "continuous": ["continuous"],
    "ci": ["ci"],
    "pump": ["pump"],
    "cost": ["cost"],
    "price": ["price"],
    "coverage": ["coverage"],
    "cost price": ["cost price"],
    "formulary": ["formulary"],
    "savings": ["savings"],
    "copay": ["copay"],
    "insurance": ["insurance"],
    "pricing": ["pricing"],
    "confirm": ["confirm"],
    "rwe": ["rwe"],
    "world": ["world"],
    "dual studies": ["dual studies"],
    "insulin degludec": ["insulin degludec"],
    "faster aspart": ["faster aspart"],
    "onset studies": ["onset studies"],
    "aace": ["aace"],
    "ace": ["ace"],
    "guidelines": ["guidelines"],
    "ada": ["ada"],
    "standards of care": ["standards of care"],
    "ellipse": ["ellipse"],
    "children": ["children"],
    "lead-1": ["lead-1"],
    "rosiglitazone": ["rosiglitazone"],
    "lead-2": ["lead-2"],
    "lead-3": ["lead-3"],
    "lead-4": ["lead-4"],
    "lead-5": ["lead-5"],
    "lead-6": ["lead-6"],
    "switch": ["switch"],
    "1860": ["1860"],
    "pratley": ["pratley"],
    "52-week data": ["52-week data"],
    "vs januvia": ["vs januvia"],
    "cover": ["cover"],
    "wfh guidelines": ["wfh guidelines"],
    "hemophilia": ["hemophilia"],
    "3rd edition": ["3rd edition"],
    "esperoct \namcp dossier": ["esperoct \namcp dossier"],
    "conclude": ["conclude"],
    "hypo": ["hypo"],
    "u300": ["u300"],
    "excursion": ["excursion"],
    "kapitza": ["kapitza"],
    "response": ["response"],
    "poster": ["poster"],
    "phase 2": ["phase 2"],
    "o'neil": ["o'neil"],
    "dose-finding": ["dose-finding","dose finding"],
    "dose-ranging": ["dose-ranging","dose ranging"],
    "exposure": ["exposure"],
    "pharmacokinetic": ["pharmacokinetic"],
    "dose-response": ["dose-response"],
    "sustain": ["sustain"],
    "gilead": ["gilead"],
    "cilofexor": ["cilofexor"],
    "firsocostat": ["firsocostat"],
    "2022": ["2022"],
    "nash": ["nash"],
    "nafld": ["nafld"],
    "non-alcoholic": ["non-alcoholic"],
    "steatohepatitis": ["steatohepatitis"],
    "fatty liver": ["fatty liver"],
    "scale": ["scale"],
    "real 1": ["real 1"],
    "treatment-naive": ["treatment-naive"],
    "sogroya": ["sogroya"],
    "aghd": ["aghd"],
    "real 4": ["real 4"],
    "ghd": ["ghd"],
    "pediatric ghd": ["pediatric ghd"],
    "weekly": ["weekly"],
    "pivotal": ["pivotal"],
    "ibt": ["ibt","intensive behavior therapy"],
    "cms": ["cms"],
    "phase 3b": ["phase 3b"],
    "step 5": ["step 5"],
    "long term": ["long term"],
    "2 year": ["2 year"],
    "104 weeks": ["104 weeks"],
    "sustained": ["sustained"],
    "step 8": ["step 8"],
    "monotherapy": ["monotherapy"],
    "sustain 10": ["sustain 10"],
    "sustain 2": ["sustain 2"],
    "sustain 3": ["sustain 3"],
    "glp-1 ra": ["glp-1 ra"],
    "sglt-2; canagliflozin": ["sglt-2; canagliflozin"],
    "invokana": ["invokana"],
    "sglt-2": ["sglt-2"],
    "add": ["add"],
    "sustain 9": ["sustain 9"],
    "sustain forte": ["sustain forte"],
    "forte": ["forte"],
    "2": ["2"],
    "2 mg": ["2 mg"],
    "ozempic 2 mg": ["ozempic 2 mg","ozempic 2mg","ozempic 2"],
    "semaglutide 2mg": ["semaglutide 2mg","semaglutide 2 mg"],
    "verma": ["verma"],
    "established cvd": ["established cvd"],
    "cv risk factors": ["cv risk factors"],
    "rewind": ["rewind"],
    "deg": ["deg"],
    "begin": ["begin"],
    "heller": ["heller"],
    "bode": ["bode"],
    "basal bolus": ["basal bolus"],
    "davies": ["davies"],
    "garber": ["garber"],
    "hollander": ["hollander"],
    "u100 vs u200": ["u100 vs u200"],
    "flex": ["flex"],
    "mathieu": ["mathieu"],
    "meneghini": ["meneghini"],
    "low volume": ["low volume"],
    "gough": ["gough"],
    "zinman": ["zinman"],
    "rodbard": ["rodbard"],
    "pediatric": ["pediatric"],
    "ideglira": ["ideglira"],
    "xultophy": ["xultophy"],
    "dual i": ["dual i"],
    "dual": ["dual"],
    "dual ii": ["dual ii"],
    "dual iii": ["dual iii"],
    "dual iv": ["dual iv"],
    "insulin naive": ["insulin naive"],
    "9": ["9"],
    "dual v": ["dual v"],
    "dual ix": ["dual ix"],
    "t2": ["t2"],
    "diabetes": ["diabetes"],
    "once": ["once"],
    "twice": ["twice"],
    "dual vii": ["dual vii"],
    "durability": ["durability"],
    "intensify": ["intensify"],
    "intensification": ["intensification"],
    "full": ["full"],
    "publications": ["publications"],
    "pubs": ["pubs"],
    "pharmacodynamics": ["pharmacodynamics"],
    "variability": ["variability"],
    "iglar": ["iglar"],
    "hypoglycemia": ["hypoglycemia"],
    "meta-analysis": ["meta-analysis"],
    "insulin glargine": ["insulin glargine"],
    "lead 1-6": ["lead 1-6"],
    "lead": ["lead"],
    "postmeal": ["postmeal"],
    "basal insulin": ["basal insulin"],
    "insulin pump": ["insulin pump"],
    "csii": ["csii"],
    "onset 4": ["onset 4"],
    "onset 5": ["onset 5"],
    "insulin aspart": ["insulin aspart"],
    "pioneer": ["pioneer"],
    "pioneer 1": ["pioneer 1","p1"],
    "placebo": ["placebo"],
    "pioneer 2": ["pioneer 2","p2"],
    "pioneer 3": ["pioneer 3","p3"],
    "pioneer 4": ["pioneer 4","p4"],
    "pioneer 5": ["pioneer 5","p5"],
    "renal": ["renal"],
    "esrd": ["esrd"],
    "dialysis": ["dialysis"],
    "gfr": ["gfr"],
    "pioneer 6": ["pioneer 6","p6"],
    "pioneer 7": ["pioneer 7","p7"],
    "flexible": ["flexible"],
    "study 2": ["study 2"],
    "study 3": ["study 3"],
    "maintenance": ["maintenance"],
    "study 1": ["study 1"],
    "prediabetes": ["prediabetes"],
    "3 years": ["3 years"],
    "long-term": ["long-term"],
    "sleep apnea": ["sleep apnea"],
    "switch-2": ["switch-2"],
    "custom": ["custom"],
    "custom nash": ["custom nash"],
    "custom letter": ["custom letter"],
    "protamine": ["protamine"],
    "mix 70/30": ["mix 70/30"],
    "death": ["death"],
    "indication": ["indication"],
    "outcomes": ["outcomes"],
    "thyroid": ["thyroid"],
    "t3": ["t3"],
    "t4": ["t4"],
    "tsh": ["tsh"],
    "demographic": ["demographic"],
    "ethnicity": ["ethnicity"],
    "race": ["race"],
    "gender": ["gender"],
    "age": ["age"],
    "baseline": ["baseline"],
    "sex": ["sex"],
    "ethnic": ["ethnic"],
    "female": ["female"],
    "male": ["male"],
    "year": ["year"],
    "hispanic": ["hispanic"],
    "latino": ["latino"],
    "white": ["white"],
    "black": ["black"],
    "african": ["african"],
    "asian": ["asian"],
    "demo": ["demo"],
    "pens": ["pens"],
    "demonstration": ["demonstration"],
    "psychiatric": ["psychiatric"],
    "anxiety": ["anxiety"],
    "depression": ["depression"],
    "mood": ["mood"],
    "devote cvd safety": ["devote cvd safety"],
    "medications concomitant other devote": ["medications concomitant other devote"],
    "alternative titration": ["alternative titration"],
    "average dose": ["average dose"],
    "titration devote": ["titration devote"],
    "smbg": ["smbg"],
    "smpg": ["smpg"],
    "glucose": ["glucose"],
    "monitor": ["monitoring","monitor"],
    "glycemic control": ["glycemic control"],
    "devote blinding blinded": ["devote blinding blinded"],
    "nocturnal": ["nocturnal"],
    "severe": ["severe"],
    "overall": ["overall"],
    "nnt": ["nnt"],
    "number needed to treat": ["number needed to treat"],
    "cv benefit": ["cv benefit"],
    "origin": ["origin"],
    "dilute": ["dilute","diluent","dilution","diluting","dilutions","dilutes","diluents","dilluting"],
    "trissel's": ["trissel's","trissel"],
    "diluted": ["diluted"],
    "compatible": ["compatible"],
    "compatibility": ["compatibility"],
    "disclaimer page": ["disclaimer page","disclaimer","disclaimer  page"],
    "stop": ["stopping","stop"],
    "rule": ["rule"],
    "degree": ["degree"],
    "percentage": ["percentage"],
    "d/c": ["d/c"],
    "tolerability": ["tolerability"],
    "efficacy": ["efficacy"],
    "icd": ["icd"],
    "employer": ["employer"],
    "identification": ["identification","id"],
    "agents": ["agents"],
    "medications": ["medications"],
    "antihyperglycemic": ["antihyperglycemic"],
    "class": ["class","classes"],
    "ominous": ["ominous"],
    "octet": ["octet"],
    "incretin": ["incretin"],
    "sterilize": ["sterilize"],
    "disinfect": ["disinfect","disinfects","disinfecting"],
    "cleanse": ["cleanse"],
    "germicide": ["germicide"],
    "infection": ["infection","infections"],
    "sanitize": ["sanitize"],
    "purify": ["purify"],
    "decontaminate": ["decontaminate","decontaminates"],
    "sterilizes": ["sterilizes"],
    "purifies": ["purifies"],
    "sterilizing": ["sterilizing"],
    "purifying": ["purifying"],
    "clean": ["clean"],
    "greater": ["greater"],
    "max": ["max"],
    "dose": ["dose","doses","dosing"],
    "titrate": ["titrate","titration","titrations","titrates"],
    "initiate": ["initiate"],
    "mg": ["mg","milligram","milligrams"],
    "recommendation": ["recommendation","recommendations"],
    "dose escalation": ["dose escalation"],
    "dose initiation": ["dose initiation"],
    "escalation": ["escalation"],
    "missed dose": ["missed dose"],
    "dosing and administration": ["dosing and administration"],
    "administration": ["administration"],
    "stopping rule": ["stopping rule"],
    "reinitiate": ["reinitiate","reinitiation","reinitiating"],
    "naïve": ["naïve"],
    "algorithim": ["algorithim"],
    "initiates": ["initiates"],
    "regimen": ["regimen"],
    "transition": ["transition"],
    "sliding": ["sliding"],
    "carb": ["carb"],
    "ratio": ["ratio"],
    "supplemental": ["supplemental"],
    "carbohydrate": ["carbohydrate"],
    "count": ["count","counting"],
    "ss": ["ss"],
    "average": ["average"],
    "mean": ["mean"],
    "fasted": ["fasted"],
    "empty stomach": ["empty stomach"],
    "qod": ["qod"],
    "other": ["other"],
    "fast": ["fast","faster"],
    "rapid": ["rapid"],
    "convert": ["convert","coverting"],
    "switching": ["switching"],
    "how to dose": ["how to dose"],
    "unbranded": ["unbranded"],
    "biologic": ["biologic"],
    "guideline": ["guideline"],
    "escalate": ["escalate"],
    "missed": ["missed"],
    "exercise": ["exercise"],
    "physical activity": ["physical activity"],
    "change": ["change"],
    "vomit": ["vomit"],
    "miss": ["miss"],
    "vomiting": ["vomiting"],
    "holiday": ["holiday"],
    "preferred time": ["preferred time"],
    "double": ["double"],
    "qd": ["qd"],
    "once-daily": ["once-daily"],
    "daily": ["daily"],
    "once-weekly": ["once-weekly"],
    "qw": ["qw"],
    "rationale": ["rationale"],
    "halflife": ["halflife","half-life","half life"],
    "steady-state": ["steady-state","steady state","steadystate"],
    "split": ["split","splitting"],
    "divide": ["divide","dividing"],
    "tiw": ["tiw"],
    "three": ["three"],
    "3": ["3"],
    "water": ["water"],
    "ounce": ["ounce"],
    "milliliter": ["milliliter"],
    "liquid": ["liquid"],
    "coffee": ["coffee"],
    "milk": ["milk"],
    "juice": ["juice"],
    "tea": ["tea"],
    "interactions": ["interactions"],
    "ddi": ["ddi"],
    "binding": ["binding"],
    "cyp 450": ["cyp 450"],
    "cytochrome": ["cytochrome"],
    "semaglutide injection": ["semaglutide injection"],
    "interaction": ["interaction"],
    "di": ["di"],
    "secretagogue": ["secretagogue"],
    "drug-drug interaction": ["drug-drug interaction","drug drug interaction"],
    "wash out period": ["wash out period"],
    "cyp": ["cyp"],
    "length": ["length"],
    "education": ["education"],
    "charitable": ["charitable"],
    "grant": ["grant"],
    "discontinue": ["discontinue"],
    "initial": ["initial"],
    "old": ["old"],
    "elderly": ["elderly"],
    "geriatric": ["geriatric"],
    "aged": ["aged"],
    "older": ["older"],
    "retired": ["retired"],
    "frail": ["frail"],
    "post-hoc": ["post-hoc"],
    "65": ["65"],
    "moa": ["moa"],
    "mechanism": ["mechanism"],
    "tfpi": ["tfpi"],
    "anti-thrombin": ["anti-thrombin"],
    "sirna": ["sirna"],
    "serpin": ["serpin"],
    "environment": ["environment","environmental"],
    "discard": ["discard"],
    "recycle": ["recycle"],
    "degradation": ["degradation"],
    "dispose": ["dispose","disposal"],
    "biodegradeable": ["biodegradeable"],
    "trash": ["trash"],
    "ub": ["ub"],
    "equivalent": ["equivalent","equivalence"],
    "substitute": ["substitute","substitution"],
    "purple book": ["purple book"],
    "orange book": ["orange book"],
    "biosimilar": ["biosimilar"],
    "interchangeable": ["interchangeable"],
    "amcp": ["amcp"],
    "estimand": ["estimand"],
    "primary": ["primary"],
    "trial product": ["trial product"],
    "treatment policy": ["treatment policy"],
    "statistical analysis": ["statistical analysis"],
    "statistics": ["statistics"],
    "elad": ["elad"],
    "aeorbic": ["aeorbic"],
    "explorer": ["explorer"],
    "explorer8": ["explorer8"],
    "fvii": ["fvii"],
    "factor vii deficiency": ["factor vii deficiency"],
    "ster": ["ster"],
    "semaglutide": ["semaglutide"],
    "fpg": ["fpg"],
    "preprandial": ["preprandial"],
    "tandem pump": ["tandem pump"],
    "medicaid": ["medicaid"],
    "flextouch": ["flextouch"],
    "prefilled": ["prefilled"],
    "airshot": ["airshot"],
    "prime": ["prime","priming"],
    "flow check": ["flow check"],
    "shot": ["shot"],
    "formulation": ["formulation"],
    "ingredient": ["ingredient","ingredients"],
    "excipient": ["excipient","excipients"],
    "glycerol": ["glycerol"],
    "phenol": ["phenol"],
    "metacresol": ["metacresol"],
    "zinc": ["zinc"],
    "disodium": ["disodium"],
    "phosphate": ["phosphate"],
    "arginine": ["arginine"],
    "l-arginine": ["l-arginine"],
    "niacinamide": ["niacinamide"],
    "b3": ["b3"],
    "fasting plasma glucose": ["fasting plasma glucose"],
    "correct": ["correct"],
    "competitor": ["competitor","competitiors","competitors"],
    "flexpen": ["flexpen"],
    "features": ["features"],
    "feature": ["feature"],
    "difference": ["difference","differences"],
    "similarity": ["similarity","similarities"],
    "vial": ["vial"],
    "syringe": ["syringe"],
    "vials": ["vials"],
    "syringes": ["syringes"],
    "gallbladder": ["gallbladder","gall-bladder","gall bladder"],
    "gallstone": ["gallstone","gallstones"],
    "cholelithiasis": ["cholelithiasis"],
    "cholecystitis": ["cholecystitis"],
    "cholecystectomy": ["cholecystectomy"],
    "gallbladder disease": ["gallbladder disease"],
    "gallbladder events": ["gallbladder events"],
    "gall": ["gall"],
    "gastroparesis": ["gastroparesis"],
    "motility": ["motility"],
    "gut": ["gut"],
    "intestine": ["intestine","intestinal"],
    "delayed": ["delayed"],
    "stomach": ["stomach"],
    "gastric emptying": ["gastric emptying"],
    "emptying": ["emptying"],
    "durable": ["durable"],
    "information": ["information"],
    "nausea": ["nausea"],
    "diarrhea": ["diarrhea"],
    "constipation": ["constipation"],
    "anti-emetic": ["anti-emetic"],
    "emesis": ["emesis"],
    "glucagon-like peptide-1": ["glucagon-like peptide-1"],
    "level": ["level","levels"],
    "gluten": ["gluten"],
    "food": ["food"],
    "allergenicity": ["allergenicity"],
    "content": ["content"],
    "gluten-free": ["gluten-free"],
    "sensitive": ["sensitive"],
    "wheat": ["wheat"],
    "celiac": ["celiac","celiac's","celiacs"],
    "intolerant": ["intolerant","intolerance"],
    "a1c": ["a1c"],
    "glycemic": ["glycemic"],
    "hemoglobin": ["hemoglobin"],
    "glycosylated": ["glycosylated"],
    "glycated": ["glycated"],
    "t2d": ["t2d"],
    "doses under 2.4 mg": ["doses under 2.4 mg"],
    "1.7 mg": ["1.7 mg"],
    "plasma": ["plasma"],
    "category": ["category"],
    "hypoglycemic": ["hypoglycemic"],
    "sus": ["sus"],
    "oral antidiabetic medications": ["oral antidiabetic medications"],
    "ppg": ["ppg"],
    "hemoglobin a1c": ["hemoglobin a1c"],
    "hcp": ["hcp"],
    "additional information required": ["additional information required"],
    "credentials": ["credentials"],
    "clarify": ["clarify"],
    "beat": ["beat"],
    "tachycardia": ["tachycardia"],
    "fibrillation": ["fibrillation"],
    "racing": ["racing"],
    "arrhythmia": ["arrhythmia"],
    "ast": ["ast"],
    "alt": ["alt"],
    "asat": ["asat"],
    "alat": ["alat"],
    "hepatic": ["hepatic"],
    "enzymes": ["enzymes"],
    "liver aspartate": ["liver aspartate"],
    "alanine": ["alanine"],
    "aminotranferase": ["aminotranferase"],
    "liver": ["liver"],
    "impairment": ["impairment"],
    "alanine aminotransferase": ["alanine aminotransferase"],
    "aspartate aminotransferase": ["aspartate aminotransferase"],
    "hepatic enzymes": ["hepatic enzymes"],
    "cirrhotic": ["cirrhotic"],
    "cirrhosis": ["cirrhosis"],
    "child-pugh": ["child-pugh"],
    "maximum": ["maximum"],
    "highest": ["highest"],
    "high": ["high"],
    "triglyceride": ["triglyceride"],
    "hypertriglyceridemia": ["hypertriglyceridemia"],
    "triglycerides": ["triglycerides"],
    "hyperlipidemia": ["hyperlipidemia"],
    "hyperlipidaemia": ["hyperlipidaemia"],
    "acute pancreatitis": ["acute pancreatitis"],
    "chronic pancreatitis": ["chronic pancreatitis"],
    "pancreatitis risk": ["pancreatitis risk"],
    "high triglycerides": ["high triglycerides"],
    "award 11": ["award 11","award-11"],
    "high-dose": ["high-dose","high dose"],
    "4.5": ["4.5"],
    "hospital": ["hospital","hospitals"],
    "inpatient": ["inpatient","inpatients"],
    "npo": ["npo"],
    "post-surgery": ["post-surgery","post surgery"],
    "post-op": ["post-op"],
    "procedure": ["procedure"],
    "colonoscopy": ["colonoscopy"],
    "potassium": ["potassium"],
    "hyperkalemia": ["hyperkalemia"],
    "hypokalemia": ["hypokalemia"],
    "hypoglycaemic": ["hypoglycaemic"],
    "hypoglycaemia": ["hypoglycaemia"],
    "nocturnal hypo": ["nocturnal hypo"],
    "insulins": ["insulins"],
    "u-300": ["u-300"],
    "low blood sugar": ["low blood sugar"],
    "sugar": ["sugar"],
    "igf-1": ["igf-1"],
    "affinity": ["affinity"],
    "cutting": ["cutting"],
    "crushing": ["crushing"],
    "swallowing": ["swallowing"],
    "tablets": ["tablets"],
    "storage": ["storage"],
    "refrigerator": ["refrigerator","refrigeration","refrigerate","refrigerates"],
    "room temperature": ["room temperature"],
    "in-use": ["in-use"],
    "inclusion criteria": ["inclusion criteria"],
    "exclusion criteria": ["exclusion criteria"],
    "inclusion": ["inclusion"],
    "exclusion": ["exclusion"],
    "leader": ["leader"],
    "emp-reg outcome": ["emp-reg outcome"],
    "control": ["control"],
    "early": ["early"],
    "hyperglycemia": ["hyperglycemia"],
    "deltoid": ["deltoid"],
    "arm": ["arm"],
    "abdomen": ["abdomen"],
    "thigh": ["thigh"],
    "injection site": ["injection site"],
    "chart": ["chart"],
    "institutions": ["institutions"],
    "long-term care": ["long-term care"],
    "ltc": ["ltc"],
    "backflow": ["backflow"],
    "contamination": ["contamination"],
    "pathogens": ["pathogens"],
    "transmission": ["transmission"],
    "blood-borne": ["blood-borne"],
    "cresol": ["cresol"],
    "preservative": ["preservative","preservatives"],
    "thimerosal": ["thimerosal"],
    "buffer": ["buffer"],
    "resistance": ["resistance","resistant"],
    "global": ["global"],
    "foreign": ["foreign"],
    "tissue factor": ["tissue factor"],
    "investigational": ["investigational"],
    "cagrisema dm": ["cagrisema dm"],
    "cagrisema": ["cagrisema"],
    "cagri": ["cagri"],
    "cagrilintide": ["cagrilintide"],
    "cagrilintide/semaglutide": ["cagrilintide/semaglutide"],
    "icodec": ["icodec"],
    "lai 287": ["lai 287"],
    "development": ["development"],
    "decitabine": ["decitabine"],
    "tetrahydrouridine": ["tetrahydrouridine"],
    "dec": ["dec"],
    "thu": ["thu"],
    "sickle cell": ["sickle cell"],
    "scd": ["scd"],
    "f4c": ["f4c"],
    "fibrosis": ["fibrosis"],
    "iss": ["iss"],
    "intravenous": ["intravenous"],
    "iv": ["iv"],
    "perioperative": ["perioperative"],
    "administer": ["administer"],
    "adsorb": ["adsorb","adsorption"],
    "infusions": ["infusions"],
    "administrate": ["administrate","administrating"],
    "lada": ["lada"],
    "latent autoimmune diabetes in adults": ["latent autoimmune diabetes in adults"],
    "type 1.5 diabetes": ["type 1.5 diabetes"],
    "latent": ["latent"],
    "1.5": ["1.5"],
    "type 1.5": ["type 1.5"],
    "latex": ["latex"],
    "rubber": ["rubber"],
    "allergen": ["allergen"],
    "nonlatex": ["nonlatex"],
    "polyisoprene": ["polyisoprene"],
    "stopper": ["stopper"],
    "biologics": ["biologics"],
    "leader trial": ["leader trial"],
    "cardiometabolic": ["cardiometabolic"],
    "secondary": ["secondary"],
    "prevention": ["prevention"],
    "effect": ["effect"],
    "retinopathy": ["retinopathy"],
    "eye": ["eye"],
    "drc": ["drc"],
    "diabetic retinopathy complications": ["diabetic retinopathy complications"],
    "diabetic retinopathy": ["diabetic retinopathy"],
    "blindness": ["blindness"],
    "blind": ["blind"],
    "vitreous hemorrhage": ["vitreous hemorrhage"],
    "retina": ["retina"],
    "visual": ["visual"],
    "microvascular": ["microvascular"],
    "60": ["60"],
    "risk factors": ["risk factors"],
    "lifestyle": ["lifestyle"],
    "diet": ["diet"],
    "adherence": ["adherence"],
    "lipids": ["lipids"],
    "hdl": ["hdl"],
    "ldl": ["ldl"],
    "cholesterol": ["cholesterol"],
    "lipid": ["lipid"],
    "vldl": ["vldl"],
    "ffa": ["ffa"],
    "lipoprotein": ["lipoprotein"],
    "fatty": ["fatty"],
    "literature": ["literature"],
    "search": ["search"],
    "literature search": ["literature search","lit search","literature  search"],
    "long": ["long"],
    "term": ["term"],
    "longterm": ["longterm","long term","long-term"],
    "long-term use": ["long-term use"],
    "scale obesity and prediabetes": ["scale obesity and prediabetes"],
    "longest": ["longest","long","longer"],
    "time": ["time"],
    "attack": ["attack"],
    "management": ["management"],
    "snac": ["snac"],
    "sodium n-[8-(2-hydroxybenzoyl) aminocaprilate]": [
        "sodium n-[8-(2-hydroxybenzoyl) aminocaprilate]"
    ],
    "sodium n-(8-[2-hydroxybenzoyl] aminocaprilate)": [
        "sodium n-(8-[2-hydroxybenzoyl] aminocaprilate)"
    ],
    "mechanism of action": ["mechanism of action"],
    "action": ["action"],
    "protraction": ["protraction"],
    "mop": ["mop"],
    "edema": ["edema","edematous"],
    "swelling": ["swelling","swollen"],
    "pitting": ["pitting"],
    "heart rate increase": ["heart rate increase"],
    "protract": ["protract"],
    "weight": ["weight"],
    "pound": ["pound"],
    "kilogram": ["kilogram"],
    "hunger": ["hunger"],
    "appetite": ["appetite"],
    "craving": ["craving"],
    "preference": ["preference"],
    "satiety": ["satiety"],
    "multihexamer": ["multihexamer"],
    "hexamer": ["hexamer"],
    "monomer": ["monomer"],
    "albumin-binding": ["albumin-binding"],
    "long-acting growth hormone": ["long-acting growth hormone"],
    "somapacitan-beco": ["somapacitan-beco"],
    "omeprazole": ["omeprazole"],
    "ppi": ["ppi"],
    "bioavailability": ["bioavailability"],
    "video": ["video"],
    "pioneer 8": ["pioneer 8"],
    "p8": ["p8"],
    "pd": ["pd"],
    "pharmacodynamic": ["pharmacodynamic"],
    "tmax": ["tmax"],
    "cmax": ["cmax"],
    "menstrual": ["menstrual","menstruation"],
    "disruption": ["disruption"],
    "disorder": ["disorder"],
    "impact": ["impact"],
    "metabolic syndrome": ["metabolic syndrome"],
    "metabolize": ["metabolize"],
    "eliminate": ["eliminate"],
    "excrete": ["excrete"],
    "mix": ["mix","mixing"],
    "neutral": ["neutral"],
    "hagedorn": ["hagedorn"],
    "mixture": ["mixture","mixtures"],
    "mtc": ["mtc"],
    "men 2": ["men 2"],
    "medullary thyroid carcinoma": ["medullary thyroid carcinoma"],
    "multiple endocrine neoplasia type 2": ["multiple endocrine neoplasia type 2"],
    "thyroid cancer": ["thyroid cancer"],
    "glp-1 ras": ["glp-1 ras"],
    "medullary": ["medullary"],
    "calcitonin parafollicular": ["calcitonin parafollicular"],
    "c-cell": ["c-cell"],
    "hyperplasia": ["hyperplasia"],
    "hepatosteatosis": ["hepatosteatosis"],
    "non-alcoholic fatty liver disease": ["non-alcoholic fatty liver disease"],
    "non-alcoholic steatohepatitis": ["non-alcoholic steatohepatitis"],
    "fatty liver disease":["fatty liver disease"],
    "non-alcoholic fatty liver disease": [
        "non-alcoholic fatty liver disease"
    ],
    "sema-nash": ["sema-nash"],
    "aga": ["aga"],
    "nafd/nash": ["nafd/nash"],
    "meal": ["meal"],
    "antiemetic": ["antiemetic"],
    "nauseous": ["nauseous"],
    "queasy": ["queasy"],
    "versus truliticy": ["versus truliticy"],
    "versus dulaglutide": ["versus dulaglutide"],
    "compared to trulicity": ["compared to trulicity"],
    "compared to dulaglutide": ["compared to dulaglutide"],
    "versus exenatide": ["versus exenatide"],
    "versus bydureon": ["versus bydureon"],
    "compared to bydureon": ["compared to bydureon"],
    "compared to exenatide": ["compared to exenatide"],
    "nedosiran": ["nedosiran"],
    "rnai": ["rnai"],
    "dicerna": ["dicerna"],
    "ph": ["ph"],
    "primary hyperoxaluria": ["primary hyperoxaluria"],
    "rare renal": ["rare renal"],
    "hyperoxaluria": ["hyperoxaluria"],
    "rna": ["rna"],
    "neonates": ["neonates"],
    "infant": ["infant"],
    "carcinogenic": ["carcinogenic"],
    "malignancy": ["malignancy","malignancies","malignant"],
    "benign": ["benign"],
    "pancreatic": ["pancreatic"],
    "neuropsychiatric": ["neuropsychiatric"],
    "suicide": ["suicide","suicidal"],
    "ideation": ["ideation"],
    "insomnia": ["insomnia"],
    "niosh": ["niosh"],
    "hazardous": ["hazardous"],
    "ndc": ["ndc"],
    "insulin aspart protamine": ["insulin aspart protamine"],
    "no data": ["no data"],
    "no": ["no"],
    "concizumab": ["concizumab"],
    "pipeline": ["pipeline"],
    "(not applicable)": ["(not applicable)"],
    "lagh": ["lagh"],
    "gh": ["gh"],
    "growth hormone": ["growth hormone"],
    "somapacitan": ["somapacitan"],
    "clinical overview": ["clinical overview"],
    "overfill": ["overfill"],
    "flexpro": ["flexpro"],
    "norditropin": ["norditropin"],
    "4 pg": ["4 pg"],
    "growth": ["growth"],
    "skytrofa": ["skytrofa"],
    "lonapegsomatropin-tcgd": ["lonapegsomatropin-tcgd"],
    "affordability": ["affordability"],
    "assistance": ["assistance"],
    "coupon": ["coupon"],
    "offerings": ["offerings"],
    "options": ["options"],
    "needles": ["needles"],
    "fit": ["fit"],
    "bd": ["bd"],
    "fits": ["fits"],
    "profile": ["profile"],
    "number": ["number"],
    "needed": ["needed"],
    "treat": ["treat"],
    "bid": ["bid"],
    "tablet": ["tablet"],
    "oasis": ["oasis"],
    "overdose": ["overdose","overdosage"],
    "medication error": ["medication error"],
    "accidental": ["accidental"],
    "intentional": ["intentional"],
    "amount": ["amount"],
    "empty": ["empty"],
    "excel": ["excel"],
    "elixa": ["elixa"],
    "harmony": ["harmony"],
    "p&t": ["p&t"],
    "pancreatic cancer": ["pancreatic cancer"],
    "adenoma": ["adenoma"],
    "pancreatic enzymes": ["pancreatic enzymes"],
    "paradigm": ["paradigm"],
    "7": ["7"],
    "alprolix": ["alprolix"],
    "rfixfc": ["rfixfc"],
    "model": ["model"],
    "novo nordisk": ["novo nordisk"],
    "partner": ["partner","partnering"],
    "speaker": ["speaker"],
    "presenter": ["presenter"],
    "passover": ["passover"],
    "kosher": ["kosher"],
    "passover; kosher": ["passover; kosher"],
    "corn": ["corn"],
    "grain": ["grain"],
    "pap": ["pap"],
    "patient assistance": ["patient assistance"],
    "pause": ["pause"],
    "te": ["te"],
    "thromb": ["thromb"],
    "pcos": ["pcos"],
    "polycystic": ["polycystic"],
    "ovary": ["ovary"],
    "syndrome": ["syndrome"],
    "pcod": ["pcod"],
    "ovarian": ["ovarian"],
    "poly": ["poly"],
    "polycystic ovary syndrome": ["polycystic ovary syndrome"],
    "scale prediabetes and obesity": ["scale prediabetes and obesity"],
    "pediatrics": ["pediatrics"],
    "child": ["child"],
    "kid": ["kid","kids"],
    "youth": ["youth"],
    "peds": ["peds"],
    "baby": ["baby","babies"],
    "neonate": ["neonate","neonatal"],
    "young": ["young"],
    "click": ["click","clicks"],
    "flow": ["flow"],
    "check": ["check"],
    "pen overview": ["pen overview"],
    "device overview": ["device overview"],
    "unmarked dosing": ["unmarked dosing"],
    "marked": ["marked"],
    "unmarked": ["unmarked"],
    "increment": ["increment"],
    "dial": ["dial"],
    "periop": ["periop"],
    "major surgery": ["major surgery"],
    "minor surgery": ["minor surgery"],
    "pets": ["pets"],
    "animals": ["animals"],
    "cats": ["cats"],
    "dogs": ["dogs"],
    "glucose-dependant": ["glucose-dependant"],
    "state": ["state"],
    "4492": ["4492"],
    "product information": ["product information"],
    "package insert": ["package insert"],
    "pilot": ["pilot"],
    "airplane": ["airplane"],
    "fly": ["fly"],
    "aviation": ["aviation"],
    "truck": ["truck"],
    "drive": ["drive","driving"],
    "cdl": ["cdl"],
    "commerce": ["commerce"],
    "machinery": ["machinery"],
    "empa-reg": ["empa-reg"],
    "phase 3a": ["phase 3a"],
    "onset": ["onset"],
    "peak": ["peak"],
    "bioequivalence": ["bioequivalence","bioequivalent"],
    "obese": ["obese"],
    "spike": ["spike"],
    "pk pd": ["pk pd"],
    "therapy": ["therapy"],
    "place": ["place"],
    "place aace ada": ["place aace ada"],
    "pneumatic": ["pneumatic"],
    "transport": ["transport"],
    "move": ["move","moving"],
    "send": ["send","sending"],
    "tube": ["tube","tubes"],
    "post-oad": ["post-oad"],
    "post-oral": ["post-oral"],
    "postprandial": ["postprandial"],
    "after": ["after"],
    "post-prandial": ["post-prandial"],
    "post": ["post"],
    "atherosclerotic": ["atherosclerotic"],
    "atherosclerosis": ["atherosclerosis"],
    "site": ["site"],
    "investigator": ["investigator"],
    "clinical trial site": ["clinical trial site"],
    "pre-diabetes": ["pre-diabetes","pre-diabetic","prediabetic"],
    "step 10": ["step 10"],
    "predrawn": ["predrawn"],
    "gdm": ["gdm"],
    "gestational": ["gestational"],
    "conception": ["conception"],
    "congenital": ["congenital"],
    "nursing": ["nursing"],
    "birth": ["birth"],
    "lactation": ["lactation"],
    "breastfeeding": ["breastfeeding","breastfeed","breastfed"],
    "maternal": ["maternal"],
    "fetal": ["fetal"],
    "fertile": ["fertile","fertility"],
    "teratogenicity": ["teratogenicity"],
    "fetus": ["fetus"],
    "press": ["press"],
    "release": ["release"],
    "product": ["product"],
    "sample": ["sample"],
    "request": ["request"],
    "prophylaxis": ["prophylaxis"],
    "prophy": ["prophy"],
    "px": ["px"],
    "inhibitors": ["inhibitors"],
    "bypassing": ["bypassing"],
    "ptc": ["ptc"],
    "papillary": ["papillary"],
    "follicular": ["follicular"],
    "qt prolongation": ["qt prolongation"],
    "ekg": ["ekg"],
    "qtc": ["qtc"],
    "thorough qt": ["thorough qt"],
    "klaus": ["klaus"],
    "lissy": ["lissy"],
    "ramadan": ["ramadan"],
    "fast": ["fast"],
    "muslim": ["muslim"],
    "islam": ["islam"],
    "cv indication": ["cv indication"],
    "cv risk reduction": ["cv risk reduction"],
    "once daily": ["once daily"],
    "week": ["week"],
    "investigating": ["investigating"],
    "produce": ["produce"],
    "synthesize": ["synthesize"],
    "production": ["production"],
    "made": ["made"],
    "make": ["make"],
    "yeast": ["yeast"],
    "saccharomyces": ["saccharomyces"],
    "cerevisiae": ["cerevisiae"],
    "manufacture": ["manufacture"],
    "manufacturing": ["manufacturing"],
    "dna": ["dna"],
    "recombinant": ["recombinant"],
    "gene": ["gene"],
    "manufactures": ["manufactures"],
    "treatment": ["treatment"],
    "reconstitute": ["reconstitute"],
    "reconstitution": ["reconstitution"],
    "saline": ["saline"],
    "histidine": ["histidine"],
    "histidine diluent": ["histidine diluent"],
    "swfi": ["swfi"],
    "concentration": ["concentration"],
    "redosing": ["redosing"],
    "half": ["half"],
    "regulatory": ["regulatory"],
    "filing": ["filing"],
    "submission": ["submission"],
    "kidney": ["kidney"],
    "egfr": ["egfr"],
    "creatinine": ["creatinine"],
    "clearance": ["clearance"],
    "crcl": ["crcl"],
    "nephrotic": ["nephrotic"],
    "impair": ["impair"],
    "end-stage": ["end-stage"],
    "glomerular": ["glomerular"],
    "transplant": ["transplant"],
    "uar": ["uar"],
    "scr": ["scr"],
    "nephropathy": ["nephropathy"],
    "renal progression": ["renal progression"],
    "improvement of renal function": ["improvement of renal function"],
    "microalbuminuria": ["microalbuminuria"],
    "albuminuria": ["albuminuria"],
    "macroalbuminuria": ["macroalbuminuria"],
    "remodel": ["remodel"],
    "educational": ["educational"],
    "material": ["material"],
    "inflammation": ["inflammation"],
    "carter": ["carter"],
    "supply": ["supply"],
    "cold": ["cold"],
    "va": ["va"],
    "veterans": ["veterans"],
    "affairs": ["affairs"],
    "shipping": ["shipping"],
    "transit": ["transit"],
    "temperature": ["temperature"],
    "dr": ["dr"],
    "retinal": ["retinal"],
    "reversal": ["reversal"],
    "fxa": ["fxa","factor xa"],
    "doac": ["doac"],
    "anticoagulant": ["anticoagulant"],
    "apixaban": ["apixaban"],
    "edoxaban": ["edoxaban"],
    "rivaroxaban": ["rivaroxaban"],
    "pcc": ["pcc"],
    "rfviia": ["rfviia"],
    "blister": ["blister"],
    "bottle": ["bottle"],
    "new packaging": ["new packaging"],
    "blue cap": ["blue cap"],
    "desiccant": ["desiccant"],
    "soul": ["soul"],
    "cardiovascular outcomes": ["cardiovascular outcomes"],
    "ongoing": ["ongoing"],
    "rybelsus": ["rybelsus"],
    "semglutide": ["semglutide"],
    "additives": ["additives"],
    "glucagon-like peptide": ["glucagon-like peptide"],
    "glutide": ["glutide"],
    "sharing": ["sharing"],
    "share": ["share","shares"],
    "microbial": ["microbial"],
    "pathogen": ["pathogen"],
    "transmit": ["transmit"],
    "transmists": ["transmists"],
    "sick": ["sick"],
    "sliding scale": ["sliding scale"],
    "ssi": ["ssi"],
    "sodium n-(8-(2-hydroxybenzoyl) amino) caprylate": [
        "sodium n-(8-(2-hydroxybenzoyl) amino) caprylate"
    ],
    "stack": ["stack","stacking"],
    "accumulate": ["accumulate","accumulation","accumulates"],
    "status": ["status"],
    "overweight": ["overweight"],
    "expiration": ["expiration"],
    "fridge": ["fridge"],
    "frozen": ["frozen"],
    "freeze": ["freeze"],
    "heat": ["heat"],
    "hot": ["hot"],
    "store": ["store"],
    "temp": ["temp"],
    "unopen": ["unopen","unponed"],
    "warm": ["warm"],
    "room": ["room"],
    "cool": ["cool"],
    "stored": ["stored"],
    "56": ["56"],
    "use": ["use"],
    "refrigerated": ["refrigerated"],
    "expired": ["expired"],
    "opened": ["opened"],
    "n8gp": ["n8gp"],
    "in vitro": ["in vitro"],
    "handling": ["handling"],
    "ship": ["ship"],
    "ambient": ["ambient"],
    "structure": ["structure"],
    "potency": ["potency"],
    "modifications": ["modify","modification","modifications"],
    "higher doses": ["higher doses"],
    "25 mg": ["25 mg"],
    "50 mg": ["50 mg"],
    "pioneer plus": ["pioneer plus"],
    "novoseven": ["novoseven"],
    "novoseven rt": ["novoseven rt"],
    "subcutaneous": ["subcutaneous"],
    "subcutaneous administration": ["subcutaneous administration"],
    "shortage": ["shortage"],
    "operation": ["operation"],
    "pre-op": ["pre-op"],
    "anesthesia": ["anesthesia"],
    "failure": ["failure"],
    "hospitalization": ["hospitalization"],
    "nyha": ["nyha"],
    "dose-dependent": ["dose-dependent"],
    "risk": ["risk"],
    "switch 1": ["switch 1"],
    "switch 2": ["switch 2"],
    "characteristics": ["characteristics"],
    "arr": ["arr"],
    "morning": ["morning"],
    "evening": ["evening"],
    "postprandial glucose": ["postprandial glucose"],
    "conversion": ["conversion"],
    "transition": ["transitioning","transition","transitions"],
    "switches": ["switches"],
    "converts": ["converts"],
    "basal-bolus": ["basal-bolus"],
    "basal-bolus glp1": ["basal-bolus glp1"],
    "glp1-ra": ["glp1-ra","glp1ra","glp-1ra"],
    "gpl1": ["gpl1"],
    "daily gh": ["daily gh"],
    "mean doses": ["mean doses"],
    "pk/pd": ["pk/pd"],
    "igf-i": ["igf-i"],
    "somatropin": ["somatropin"],
    "lorcaserin": ["lorcaserin"],
    "belviq": ["belviq"],
    "premix": ["premix"],
    "glucagon-like": ["glucagon-like"],
    "conversion. liraglutide": ["conversion. liraglutide"],
    "anti-obesity medication": ["anti-obesity medication"],
    "glp": ["glp"],
    "dose conversion": ["dose conversion"],
    "soliqua": ["soliqua"],
    "100/33": ["100/33"],
    "lixilan": ["lixilan"],
    "suggestion": ["suggestion"],
    "hypothyroidism": ["hypothyroidism"],
    "hyperthyroidism": ["hyperthyroidism"],
    "goiter": ["goiter"],
    "nodules": ["nodules"],
    "thyroidectomy": ["thyroidectomy"],
    "papillary thyroid carcinoma": ["papillary thyroid carcinoma"],
    "men2": ["men2"],
    "multiple endocrine neoplasia syndrome type 2": [
        "multiple endocrine neoplasia syndrome type 2"
    ],
    "nodule": ["nodule"],
    "hypothyroid": ["hypothyroid"],
    "thyroid conditions": ["thyroid conditions"],
    "pre-existing thyroid conditions": ["pre-existing thyroid conditions"],
    "thyroid follicular tissue": ["thyroid follicular tissue"],
    "thyroid gland": ["thyroid gland"],
    "thyroid hormones": ["thyroid hormones"],
    "men": ["men"],
    "neoplasia": ["neoplasia"],
    "thyroid neoplasms": ["thyroid neoplasms"],
    "calcitonin": ["calcitonin"],
    "follicular thyroid cancer": ["follicular thyroid cancer"],
    "ftc": ["ftc"],
    "tid": ["tid"],
    "timing": ["timing"],
    "pks": ["pks"],
    "pds": ["pds"],
    "time of dosing comparison": ["time of dosing comparison"],
    "sunshine": ["sunshine"],
    "act": ["act"],
    "transparency": ["transparency"],
    "payments": ["payments"],
    "solid organ transplant": ["solid organ transplant"],
    "kidney transplant": ["kidney transplant"],
    "liver transplant": ["liver transplant"],
    "transplant eligibility": ["transplant eligibility"],
    "qualify for transplant": ["qualify for transplant"],
    "treat-to-target": ["treat-to-target"],
    "type 1": ["type 1","type-1","type1","typei","type i","type-i"],
    "t1d": ["t1d","t1dm","tidm","type 1 diabetes"],
    "t1dm":["t1dm","tidm","type 1 diabetes mellitus"],
    "off-label": ["off-label"],
    "u500": ["u500","u-500","u 500"],
    "500": ["500"],
    "concentrated": ["concentrated"],
    "500u/ml": ["500u/ml"],
    "contact": ["contact"],
    "reach": ["reach"],
    "mentor": ["mentor"],
    "coronavirus": ["coronavirus"],
    "covid-19": ["covid-19"],
    "covid": ["covid"],
    "coagulopathy": ["coagulopathy"],
    "injections": ["injections"],
    "0.5": ["0.5"],
    "past": ["past"],
    "variation": ["variation"],
    "meal-time": ["meal-time"],
    "duration-6": ["duration-6"],
    "75/25": ["75/25"],
    "70/30": ["70/30"],
    "human": ["human"],
    "novolin": ["novolin"],
    "rhi": ["rhi"],
    "humulin": ["humulin"],
    "regular": ["regular"],
    "xultophy 100/3.6": ["xultophy 100/3.6"],
    "mounjaro": ["mounjaro"],
    "surpass": ["surpass"],
    "gip": ["gip"],
    "lomaira": ["lomaira"],
    "eli": ["eli"],
    "gate": ["gate"],
    "genentech": ["genentech"],
    "genotropin": ["genotropin"],
    "humatrope": ["humatrope"],
    "lilly": ["lilly"],
    "nutropin": ["nutropin"],
    "omnitrope": ["omnitrope"],
    "pfizer": ["pfizer"],
    "protropin": ["protropin"],
    "saizen": ["saizen"],
    "sandoz": ["sandoz"],
    "serono": ["serono"],
    "serostim": ["serostim"],
    "tev-tropin": ["tev-tropin"],
    "zorbtive": ["zorbtive"],
    "valtropin": ["valtropin"],
    "semaglutide sq": ["semaglutide sq"],
    "semaglutide sc": ["semaglutide sc"],
    "semaglutide subcutaneous": ["semaglutide subcutaneous"],
    "semaglutide comparison": ["semaglutide comparison"],
    "2.0 mg": ["2.0 mg"],
    "actrapid": ["actrapid"],
    "sevenfact": ["sevenfact"],
    "factor viia": ["factor viia"],
    "surmount": ["surmount"],
    "tirzepatide": ["tirzepatide"],
    "award-6": ["award-6"],
    "award": ["award"],
    "admelog": ["admelog"],
    "afrezza": ["afrezza"],
    "inhaled": ["inhaled"],
    "inhale": ["inhale"],
    "aabc": ["aabc"],
    "fiasp vs novolog": ["fiasp vs novolog"],
    "aspart comparison": ["aspart comparison"],
    "junior": ["junior"],
    "jr": ["jr"],
    "stimulation": ["stimulation"],
    "diagnosis": ["diagnosis"],
    "tolerance": ["tolerance"],
    "waist": ["waist"],
    "inches": ["inches"],
    "circumference": ["circumference"],
    "bmi": ["bmi"],
    "kg": ["kg"],
    "lb": ["lb"],
    "lbs": ["lbs"],
    "1.7": ["1.7"],
    "1.0": ["1.0"],
    "regain": ["regain"],
    "body weight": ["body weight"],
    "body mass index": ["body mass index"],
    "lower doses": ["lower doses"],
    "weight reduction": ["weight reduction"],
    "phase 2 dose-finding": ["phase 2 dose-finding"],
    "scale diabetes": ["scale diabetes"],
    "levemir vs tresiba": ["levemir vs tresiba"],
    "glargine u-300": ["glargine u-300"],
    "mechansim": ["mechansim"],
    "theory": ["theory"],
    "a1c comparison": ["a1c comparison"],
    "reductions": ["reductions"],
    "theories": ["theories"],
    "withdrawal": ["withdrawal"],
    "zn": ["zn"],
}


In [9]:
import spacy
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# def lemmatize_text(text):
#     doc = nlp(text)
#     lemmatized_text = " ".join([token.lemma_ for token in doc])
#     return lemmatized_text

# text = "Provider has a patient with type 2 diabetes who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
# lemmatized_text = lemmatize_text(text)
# print(lemmatized_text)
from textblob import Word

def stem_text(text):
    words = text.split()
    stemmed_words = [Word(word).stem() for word in words]
    stemmed_text = " ".join(stemmed_words)
    return stemmed_text

text = "Provider has a patient with type 2 diabetes who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
stemmed_text = stem_text(text)
print(stemmed_text)

provid ha a patient with type 2 diabet who wa diagnos with a benign tumor on the pancreas. she want to find out if we have ani studi that includ patient with the same condition, and is it safe to start them on ozempic?


In [6]:
!pip install textblob

     -------------------------------------- 636.8/636.8 kB 5.0 MB/s eta 0:00:00


In [12]:
import stanza

def stem_text(text):
    stanza.download('en')  # Download the English models

    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')  # Initialize the pipeline

    doc = nlp(text)
    stemmed_text = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
    return stemmed_text

text = "Provider has a patient with type 2 diabetes who was diagnosed with a benign tumor on the pancreas. She wanted to find out if we have any studies that included patients with the same condition, and is it safe to start them on Ozempic?"
stemmed_text = stem_text(text)
print(stemmed_text)

2023-06-05 10:21:07 INFO: Downloading default packages for language: en (English) ...


2023-06-05 10:23:23 INFO: Finished downloading models and saved to C:\Users\Owner\stanza_resources.
2023-06-05 10:23:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-06-05 10:23:24 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-06-05 10:23:25 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2023-06-05 10:23:25 INFO: Using device: cpu
2023-06-05 10:23:25 INFO: Loading: tokenize
2023-06-05 10:23:25 INFO: Loading: pos
2023-06-05 10:23:26 INFO: Loading: lemma
2023-06-05 10:23:26 INFO: Loading: depparse
2023-06-05 10:23:27 INFO: Done loading processors!


provider have a patient with type 2 diabete who be diagnose with a benign tumor on the pancrea . she want to find out if we have any study that include patient with the same condition , and be it safe to start they on Ozempic ?


In [11]:
!pip install stanza

     -------------------------------------- 802.5/802.5 kB 4.6 MB/s eta 0:00:00
     -------------------------------------- 353.7/353.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 172.4/172.4 MB 2.9 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350810 sha256=969d732939157e7f697a04e7abe1e2f89669408633485ac5f8dfc8094f9d5b40
  Stored in directory: c:\users\owner\appdata\local\pip\cache\wheels\d5\c5\45\6c3b2e538c10e9667495d6bff243ee0a296c3c246478007df3
Successfully built emoji
